## Import libs

In [1]:
import pandas as pd
import numpy as np

## Import data

In [8]:
bg = pd.read_feather('../Board-Games/data/full/bgg_boardgames.feather')
# ratings = pd.read_feather('./data/full/bgg_ratings_full.feather')
bg.boardgame_id = bg.boardgame_id.astype('int')
# bg.minplayers = bg.minplayers.astype('int')
# bg.maxplayers = bg.maxplayers.astype('int')
# bg.minplaytime = bg.minplaytime.astype('int')
# bg.maxplaytime = bg.maxplaytime.astype('int')
# ratings = ratings[['nickname', 'boardgame_id', 'rating']].merge(bg[['boardgame_id', 'title']], 
#                                                                 on='boardgame_id', how='inner')[['nickname', 
#                                                                                        'title', 'rating']]

## Logic

To begin with, we will implement the simplest version of the recommendations for interacting with the chatbot, both at a cold-start and for users with an existing history.

In either case, it's worth getting input for making recommendations:
- Number of players:
    - solo
    - for two
    - 3-4
    - 5+

- Game duration:
    - up to 30 minutes
    - 30 - 90 minutes
    - 1.5 - 3 hours
    - more than 3 hours
    
- How long ago the game was released:
    - up to 3 years ago (newest)
    - 3-7 years ago (recent)
    - older than 7 years ago (older)
    - before 2007 (oldschool games)
    
- Preferred genres (?):
    - ...
    - ...
    
And later with these filters we will recommend games with the highest rank on BGG